In [5]:
from pathlib import Path

In [6]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import torch
from torch import nn
from torch import optim
from torch.nn import functional as F
from torch.optim.lr_scheduler import _LRScheduler
from torch.utils.data import TensorDataset, DataLoader

In [7]:
seed = 1
np.random.seed(seed)

In [8]:
# ROOT = Path.cwd().parent/'input'
ROOT = Path.home()/'data'/'careercon2019'

In [9]:
SAMPLE = ROOT/'sample_submission.csv'
TRAIN = ROOT/'X_train.csv'
TARGET = ROOT/'y_train.csv'
TEST = ROOT/'X_test.csv'

ID_COLS = ['series_id', 'measurement_number']

x_cols = {
    'series_id': np.uint32,
    'measurement_number': np.uint32,
    'orientation_X': np.float32,
    'orientation_Y': np.float32,
    'orientation_Z': np.float32,
    'orientation_W': np.float32,
    'angular_velocity_X': np.float32,
    'angular_velocity_Y': np.float32,
    'angular_velocity_Z': np.float32,
    'linear_acceleration_X': np.float32,
    'linear_acceleration_Y': np.float32,
    'linear_acceleration_Z': np.float32
}

y_cols = {
    'series_id': np.uint32,
    'group_id': np.uint32,
    'surface': str
}

In [10]:
x_trn = pd.read_csv(TRAIN, usecols=x_cols.keys(), dtype=x_cols)
x_tst = pd.read_csv(TEST, usecols=x_cols.keys(), dtype=x_cols)
y_trn = pd.read_csv(TARGET, usecols=y_cols.keys(), dtype=y_cols)

In [11]:
def add_euler_angles(df):
    """Adds Euler angles features to the dataset."""
    
    x, y, z, w = [df[f'orientation_{s}'] for s in list('XYZW')]
    nx, ny, nz = quaternion_to_euler(x, y, z, w)
    df['euler_X'] = nx
    df['euler_Y'] = ny
    df['euler_Z'] = nz
    return df

In [12]:
def quaternion_to_euler(x, y, z, w):
    """Converts quaternion values into Euler angles (roll, pitch and yaw)."""
    
    t0 = 2.0*(w*x + y*z)
    t1 = 1.0 - 2.0*(x*x + y*y)
    X = np.arctan2(t0, t1)
    
    t2 = np.clip(2.0*(w*y - z*x), -1, 1)
    Y = np.arcsin(t2)
    
    t3 = 2.0*(w*z + x*y)
    t4 = 1.0 - 2.0*(y*y + z*z)
    Z = np.arctan2(t3, t4)
    
    return X, Y, Z

In [13]:
def startswith(df, prefix):
    return df.columns[df.columns.str.startswith(prefix)].tolist()

In [14]:
trn_sz, tst_sz = x_trn.series_id.nunique(), x_tst.series_id.nunique()
print(f'Number of series: {trn_sz} train, {tst_sz} test')

Number of series: 3810 train, 3816 test


In [15]:
x_tst['series_id'] += len(x_trn)

In [16]:
data = pd.concat([x_trn, x_tst], axis=0).reset_index(drop=True)

In [17]:
data = add_euler_angles(data)

In [18]:
data = data.drop(columns=['measurement_number'] + startswith(data, 'orient'))

In [19]:
data.sample(5).T

,243558,119111,928918,815480,249925
series_id,1902.000000,930.000000,491127.000000,490240.000000,1952.000000
angular_velocity_X,-0.027397,0.023170,0.129640,0.037026,-0.017861
angular_velocity_Y,-0.038235,-0.071596,-0.003962,-0.011218,-0.045124
angular_velocity_Z,0.141130,0.110860,0.074504,-0.085403,-0.091425
linear_acceleration_X,0.546210,1.192700,7.679400,1.004200,-2.370200
linear_acceleration_Y,1.838000,1.558500,-0.908840,2.135700,2.500600
linear_acceleration_Z,-9.821200,-10.435000,-9.836400,-11.972000,-9.029200
euler_X,2.842510,2.837526,2.847498,2.842665,2.841162
euler_Y,-0.009827,-0.012305,-0.018179,-0.017390,-0.011367
euler_Z,-1.595382,-1.796759,1.863175,-3.101702,-0.247120


In [20]:
euler_cols = startswith(data, 'euler')
linear_cols = startswith(data, 'linear') 
angular_cols = startswith(data, 'angular')

In [21]:
def abs_fft(arr): return np.abs(np.fft.rfft(arr))

In [22]:
def zero_mean(x): return x - x.mean()

In [23]:
def zscore(x): return (x - x.mean())/x.std()

In [24]:
groups = data.groupby('series_id')

In [25]:
data = pd.concat([
    groups[euler_cols].diff().fillna(0),
    groups[linear_cols].transform(zero_mean),
    groups[angular_cols].transform(zero_mean)
], axis=1, sort=False)

In [26]:
fft_data = (
    groups[linear_cols + angular_cols]
    .apply(lambda df: df.apply(abs_fft, axis=0))
    .reset_index('series_id', drop=True))

In [27]:
seq_len = 128
fft_seq_len = seq_len//2 + 1
num_classes = 9

In [28]:
# Shape of array: (batch, features, time dimension)
raw_arr = data.values.reshape([trn_sz + tst_sz, len(data.columns),  seq_len])
fft_arr = fft_data.values.reshape([trn_sz + tst_sz, len(fft_data.columns), fft_seq_len])
print(f'Prepared datasets shapes: {raw_arr.shape} raw, {fft_arr.shape} fft')

Prepared datasets shapes: (7626, 9, 128) raw, (7626, 6, 65) fft


In [32]:
feat_arr = np.load('/home/ck/data/careercon2019/tmp/feat.npy').transpose(0, 2, 1)

In [35]:
np.allclose(raw_arr, feat_arr)

False

In [26]:
enc = LabelEncoder().fit(y_trn['surface'])
target = list(enc.transform(y_trn['surface']))
target += [0] * tst_sz
target = np.array(target)
assert len(target) == trn_sz + tst_sz

In [27]:
def create_datasets(data, target, train_size, valid_pct=0.1, seed=None):
    raw, fft = data
    assert len(raw) == len(fft)
    sz = train_size
    idx = np.arange(sz)
    trn_idx, val_idx = train_test_split(
        idx, test_size=valid_pct, random_state=seed)
    trn_ds = TensorDataset(
        torch.tensor(raw[:sz][trn_idx]).float(), 
        torch.tensor(fft[:sz][trn_idx]).float(), 
        torch.tensor(target[:sz][trn_idx]).long())
    val_ds = TensorDataset(
        torch.tensor(raw[:sz][val_idx]).float(), 
        torch.tensor(fft[:sz][val_idx]).float(), 
        torch.tensor(target[:sz][val_idx]).long())
    tst_ds = TensorDataset(
        torch.tensor(raw[sz:]).float(), 
        torch.tensor(fft[sz:]).float(), 
        torch.tensor(target[sz:]).long())
    return trn_ds, val_ds, tst_ds

In [83]:
def create_loaders(data, bs=128, jobs=0):
    trn_ds, val_ds, tst_ds = data
    trn_dl = DataLoader(trn_ds, batch_size=bs, shuffle=True, num_workers=jobs)
    val_dl = DataLoader(val_ds, batch_size=bs, shuffle=False, num_workers=jobs)
    tst_dl = DataLoader(tst_ds, batch_size=bs, shuffle=False, num_workers=jobs)
    return trn_dl, val_dl, tst_dl

In [84]:
datasets = create_datasets((raw_arr, fft_arr), target, trn_sz, seed=seed)

In [85]:
def bn1d_drop_layer(layers, n_outputs, drop=None, bn=True, activ=nn.ReLU):
    """Adds batchnorm, dropout, and/or activation layer(s) 
    to the list of layers.
    """
    if bn:
        layers += [nn.BatchNorm1d(n_outputs)]
    if activ is not None:
        layers += [activ()]
    if drop and 0.0 < drop < 1.0:
        layers += [nn.Dropout(drop)]
    return layers

In [86]:
def conv1d(ni, no, kernel=3, stride=1, pad=0,
           drop=None, bn=True, activ=nn.ReLU):
    """A 1-d convolutional layer with few additional layers on top of it."""
    
    layers = [nn.Conv1d(ni, no, kernel, stride, pad, bias=not bn)]
    return bn1d_drop_layer(layers, no, drop, bn, activ)

In [99]:
def fc(ni, no, drop=None, bn=True, activ=nn.ReLU):
    """A fully connected layer with few additional layers on top of it."""
    
    layers = [nn.Linear(ni, no, bias=not bn)]
    return bn1d_drop_layer(layers, no, drop, bn, activ)

In [100]:
class SeparableConv1d(nn.Module):
    def __init__(self, ni, no):
        super().__init__()
        self.depthwise = nn.Conv1d(ni, ni, kernel_size=3, padding=1, groups=ni)
        self.pointwise = nn.Conv1d(ni, no, kernel_size=1)

    def forward(self, x):
        return self.pointwise(self.depthwise(x))

In [101]:
class Flatten(nn.Module):
    """Converts N-dimensional tensor into 'flat' one."""

    def __init__(self, keep_batch_dim=True):
        super().__init__()
        self.keep_batch_dim = keep_batch_dim

    def forward(self, x):
        if self.keep_batch_dim:
            return x.view(x.size(0), -1)
        return x.view(-1)

In [102]:
# class TimeSeriesEncoder(nn.Module):
#     def __init__(self, ni, drop=.5):
#         super().__init__()
#         self.layers = nn.ModuleList([
#             *conv1d( ni,  32, drop=drop),
#             *conv1d( 32,  64, drop=drop),
#             *conv1d( 64, 128, drop=drop),
#             *conv1d(128, 256, drop=drop),
#             nn.AdaptiveAvgPool1d(1),
#             Flatten(),
#             nn.Dropout(drop),
#             *fc(256, 64, drop=drop),
#             *fc( 64, 64)
#         ])
    
#     def forward(self, x):
#         for layer in self.layers:
#             x = layer(x)
#         return x

In [103]:
class Debugger(nn.Module):
    def forward(self, x):
        return x

In [104]:
# class TimeSeriesEncoder(nn.Module):
#     def __init__(self, ni, drop=.5):
#         super().__init__()
#         self.layers = nn.ModuleList([
#             nn.Conv1d(ni, 32, 8, 2, padding=3),
#             nn.ReLU(inplace=True),
#             nn.Dropout(drop),
#             nn.Conv1d(32, 64, 8, 4, padding=2),
#             nn.ReLU(inplace=True),
#             nn.Dropout(drop),
#             nn.Conv1d(64, 128, 8, 4, padding=2),
#             nn.ReLU(inplace=True),
#             nn.Dropout(drop),
#             nn.Conv1d(128, 256, 8, 4, padding=2),
#             nn.ReLU(inplace=True),
#             Flatten(),
#             nn.Dropout(drop),
#             nn.Linear(256, 64),
#             nn.ReLU(inplace=True),
#             nn.Dropout(drop),
#             nn.Linear(64, 64),
#             nn.ReLU(inplace=True)
#         ])
        
#     def forward(self, x):
#         for layer in self.layers:
#             x = layer(x)
#         return x

In [166]:
class Conv1d(nn.Module):
    def __init__(self, ni, no, kernel, stride=1, pad=0, drop=.5):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Conv1d(ni, no, kernel, stride, padding=pad),
            nn.ReLU(inplace=True),
            nn.Dropout(drop))
        
    def forward(self, x): 
        return self.layers(x)

In [178]:
class TimeSeriesEncoder(nn.Module):
    def __init__(self, conv, top, drop=.5):
        super().__init__()
        layers = []
        for ni, no, kernel, stride, pad in conv:
            layers.append(Conv1d(ni, no, kernel, stride, pad, drop))
            # layers += [
            #    nn.Conv1d(ni, no, kernel, stride, padding=pad),
            #    nn.ReLU(inplace=True),
            #    nn.Dropout(drop)]
        layers.append(Flatten())
        _, in_size, *_ = conv[-1]
        for out_size in top:
            layers.append(nn.Dropout(drop))
            layers.append(nn.Linear(in_size, out_size))
            in_size = out_size
        self.layers = nn.ModuleList(layers)
        
    def forward(self, x): 
        for layer in self.layers:
            x = layer(x)
        return x

In [179]:
class Classifier(nn.Module):
    def __init__(self, raw_ni, fft_ni, no):
        super().__init__()
        self.raw = TimeSeriesEncoder(conv=[
            (raw_ni,  32, 8, 2, 3),
            (    32,  64, 8, 4, 2),
            (    64, 128, 8, 4, 2),
            (   128, 256, 8, 4, 2)
        ], top=[64, 64])
        self.fft = TimeSeriesEncoder(conv=[
            (fft_ni,  32, 8, 2, 4),
            (    32,  64, 8, 2, 4),
            (    64, 128, 8, 4, 4),
            (   128, 128, 8, 4, 4),
            (   128, 256, 8, 2, 3)
        ], top=[64, 64])
        self.out = nn.Sequential(
            nn.Linear(128, 64),
            nn.ReLU(inplace=True),
            nn.Linear(64, no)
        )
    
    def forward(self, t_raw, t_fft):
        raw_out = self.raw(t_raw)
        fft_out = self.fft(t_fft)
        t_in = torch.cat([raw_out, fft_out], dim=1)
        out = self.out(t_in)
        return out

In [180]:
class CyclicLR(_LRScheduler):
    
    def __init__(self, optimizer, schedule, last_epoch=-1):
        assert callable(schedule)
        self.schedule = schedule
        super().__init__(optimizer, last_epoch)

    def get_lr(self):
        return [self.schedule(self.last_epoch, lr) for lr in self.base_lrs]

In [181]:
def cosine(t_max, eta_min=0):
    
    def scheduler(epoch, base_lr):
        t = epoch % t_max
        return eta_min + (base_lr - eta_min)*(1 + np.cos(np.pi*t/t_max))/2
    
    return scheduler

In [182]:
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')

In [184]:
raw_feat = raw_arr.shape[1]
fft_feat = fft_arr.shape[1]

trn_dl, val_dl, tst_dl = create_loaders(datasets, bs=256)

lr = 0.001
n_epochs = 3000
iterations_per_epoch = len(trn_dl)
best_acc = 0
patience, trials = 500, 0

model = Classifier(raw_feat, fft_feat, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
opt = torch.optim.Adam(model.parameters(), lr=lr)
sched = CyclicLR(opt, cosine(t_max=iterations_per_epoch * 2, eta_min=lr/100))

print('Start model training')

for epoch in range(1, n_epochs + 1):
    
    model.train()
    for i, batch in enumerate(trn_dl):
        x_raw, x_fft, y_batch = [t.to(device) for t in batch]
        sched.step()
        opt.zero_grad()
        out = model(x_raw, x_fft)
        loss = criterion(out, y_batch)
        loss.backward()
        opt.step()
    
    model.eval()
    correct, total = 0, 0
    for batch in val_dl:
        x_raw, x_fft, y_batch = [t.to(device) for t in batch]
        out = model(x_raw, x_fft)
        preds = F.log_softmax(out, dim=1).argmax(dim=1)
        total += y_batch.size(0)
        correct += (preds == y_batch).sum().item()
    
    acc = correct / total

    if epoch % 5 == 0:
        print(f'Epoch: {epoch:3d}. Loss: {loss.item():.4f}. Acc.: {acc:2.2%}')

    if acc > best_acc:
        trials = 0
        best_acc = acc
        torch.save(model.state_dict(), 'best.pth')
        print(f'Epoch {epoch} best model saved with accuracy: {best_acc:2.2%}')
    else:
        trials += 1
        if trials >= patience:
            print(f'Early stopping on epoch {epoch}')
            break

Start model training
Epoch 1 best model saved with accuracy: 21.26%
Epoch 4 best model saved with accuracy: 23.62%
Epoch:   5. Loss: 2.0629. Acc.: 22.57%
Epoch 8 best model saved with accuracy: 26.77%
Epoch:  10. Loss: 1.7151. Acc.: 28.35%
Epoch 10 best model saved with accuracy: 28.35%
Epoch 12 best model saved with accuracy: 29.66%
Epoch 13 best model saved with accuracy: 31.23%
Epoch:  15. Loss: 1.6958. Acc.: 32.81%
Epoch 15 best model saved with accuracy: 32.81%
Epoch 19 best model saved with accuracy: 34.38%
Epoch:  20. Loss: 1.5414. Acc.: 32.55%
Epoch:  25. Loss: 1.5464. Acc.: 29.66%
Epoch:  30. Loss: 1.6755. Acc.: 31.76%
Epoch:  35. Loss: 1.6121. Acc.: 27.56%
Epoch:  40. Loss: 1.5160. Acc.: 28.08%
Epoch:  45. Loss: 1.4838. Acc.: 29.40%
Epoch:  50. Loss: 1.5336. Acc.: 29.92%
Epoch:  55. Loss: 1.5008. Acc.: 29.13%
Epoch:  60. Loss: 1.4944. Acc.: 30.71%
Epoch:  65. Loss: 1.3241. Acc.: 30.97%
Epoch:  70. Loss: 1.4255. Acc.: 30.45%
Epoch:  75. Loss: 1.5957. Acc.: 35.43%
Epoch 75 best

Epoch: 930. Loss: 0.7082. Acc.: 39.11%
Epoch: 935. Loss: 0.7448. Acc.: 39.63%
Epoch: 940. Loss: 0.6917. Acc.: 42.78%
Epoch: 945. Loss: 0.7073. Acc.: 42.26%
Epoch: 950. Loss: 0.7643. Acc.: 37.80%
Epoch: 955. Loss: 0.8582. Acc.: 39.90%
Epoch: 960. Loss: 0.5466. Acc.: 40.42%
Epoch: 965. Loss: 0.7807. Acc.: 38.06%
Epoch: 970. Loss: 0.5902. Acc.: 37.27%
Epoch: 975. Loss: 0.6724. Acc.: 37.53%
Epoch: 980. Loss: 0.6606. Acc.: 39.37%
Epoch: 985. Loss: 0.6720. Acc.: 39.37%
Epoch: 990. Loss: 0.7509. Acc.: 41.21%
Epoch: 995. Loss: 0.7154. Acc.: 41.73%
Epoch: 1000. Loss: 0.6033. Acc.: 38.06%
Epoch: 1005. Loss: 0.6504. Acc.: 38.85%
Epoch: 1010. Loss: 0.6947. Acc.: 38.58%
Epoch: 1015. Loss: 0.6506. Acc.: 35.96%
Epoch: 1020. Loss: 0.7027. Acc.: 37.80%
Epoch: 1025. Loss: 0.7430. Acc.: 38.06%
Epoch: 1030. Loss: 0.7173. Acc.: 39.11%
Epoch: 1035. Loss: 0.8282. Acc.: 41.73%
Epoch: 1040. Loss: 0.6078. Acc.: 39.11%
Epoch: 1045. Loss: 0.6722. Acc.: 40.42%
Epoch: 1050. Loss: 0.7703. Acc.: 38.58%
Epoch: 1055. L